In [1]:
%load_ext autoreload
%autoreload 2

Set up the gym and wrap a monitor around it that will periodically record movies as it learns

In [2]:
import gym
from surface_seg.envs.mcs_env import MCSEnv
import gym.wrappers

# MCS_gym = MCSEnv()
MCS_gym = gym.make('surface_seg-v0')
MCS_gym = gym.wrappers.Monitor(MCS_gym, "./vid", force=True)

print('Starting energy = %1.2f eV'%MCS_gym.atoms.get_potential_energy())

Starting energy = 2.49 eV


Set up the gym and agent in tensorforce

In [3]:
import tensorforce 
from tensorforce.agents import Agent


env = tensorforce.environments.OpenAIGym(MCS_gym,
                                                 max_episode_timesteps=200,
                                                visualize=False)

agent = Agent.create(
    agent='dqn', 
    environment=env, 
    batch_size=10, 
    learning_rate=1e-3,
    memory = 40000,
    max_episode_timesteps = 200,
    exploration=0.5)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


Run the DRL method for a while

In [4]:
from tensorforce.execution import Runner

runner = Runner(
    agent=agent,
    environment=env,
    max_episode_timesteps=200,
)

runner.run(num_episodes=1000)

# runner.run(num_episodes=10, evaluation=True)

runner.close()

Episodes:   0%|          | 0/1000 [00:00, reward=0.00, ts/ep=0, sec/ep=0.00, ms/ts=0.0, agent=0.0%]/home/junwoony/miniconda3/envs/arpae/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Env returned None on render(). Disabling further rendering for video recorder by marking as disabled: path=/home/junwoony/Desktop/surface_seg/examples/vid/openaigym.video.0.21665.video000000.mp4 metadata_path=/home/junwoony/Desktop/surface_seg/examples/vid/openaigym.video.0.21665.video000000.meta.json
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))
Episodes:   0%|          | 1/1000 [00:09, reward=-0.07, ts/ep=200, sec/ep=9.32, ms/ts=46.6, agent=24.0%]/home/junwoony/miniconda3/envs/arpae/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Env returned None on render(). Disabling further rendering for video recorder by marking as disabled: path=/home/junwoony/Desktop/surface_seg/examples/vid/openaigym.video.0.21665.video000001.mp4 metadata_path=/home/junwoony

Traceback (most recent call last):
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-fb2386194ba1>", line 9, in <module>
    runner.run(num_episodes=1000)
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.8/site-packages/tensorforce/execution/runner.py", line 469, in run
    observation = self.environments[n].receive_execute()
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.8/site-packages/tensorforce/environments/environment.py", line 325, in receive_execute
    states, terminal, reward = self.execute(actions=self._actions)
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.8/site-packages/tensorforce/environments/environment.py", line 366, in execute
    states, terminal, reward = self.environment.execute(actions=actions)
  File "/home/junwoony/miniconda3/envs/arpae/lib/python3.8/site-packages/tensorforce

KeyboardInterrupt: 

# Example of how to render the environment in a jupyter cell

In [ ]:
env = MCS_gym
from matplotlib import pyplot as plt
%matplotlib inline

from IPython import display

img = plt.imshow(env.render(mode='rgb_array')) # only call this once
plt.axis('off')
for _ in range(100):
    img.set_data(env.render(mode='rgb_array')) # just update the data
    display.display(plt.gcf())
    display.clear_output(wait=True)
    action = env.action_space.sample()
    env.step(action)